# LlamaCloud Client SDK: Integrating LlamaHub Loaders

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/client_sdk/llamahub_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial shows you how to use LlamaHub loaders to insert documents with LlamaCloud, with the help of the lower-level LlamaCloud Client SDK.

In this example, we use the [Firecrawl web page reader](https://www.firecrawl.dev/) in order to load a web page document, and feed it into our LlamaCloud pipeline.

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install llama-index
!pip install llama-cloud
!pip install llama-index-readers-web firecrawl-py

## Setup

Here we setup our environment variables, client, and load data using the Firecrawl loader available on LlamaHub.

The Firecrawl loader is available as part of our `llama-index-readers-web` package.

In [2]:
import os

os.environ["LLAMA_CLOUD_BASE_URL"] = "https://api.cloud.llamaindex.ai"

In [18]:
os.environ["LLAMA_CLOUD_API_KEY"] = "<LLAMA_CLOUD_API_KEY>"
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [19]:
FIRECRAWL_API_KEY = "<FIRECRAWL_API_KEY>"

#### Load Data

In [20]:
from llama_index.readers.web import FireCrawlWebReader

# using firecrawl to crawl a website
firecrawl_reader = FireCrawlWebReader(
    api_key=FIRECRAWL_API_KEY,
    mode="scrape",  # Choose between "crawl" and "scrape" for single page scraping
    # params={"additional": "parameters"},  # Optional additional parameters
)

In [36]:
# Load documents from a single page URL
documents = firecrawl_reader.load_data(url="https://www.oreilly.com/radar/what-we-learned-from-a-year-of-building-with-llms-part-i/")

In [ ]:
print(documents[0].get_content())

#### Setup Index

Please setup an empty index. You can either do this through the UI or [programmatically](https://docs.cloud.llamaindex.ai/llamacloud/guides/framework_integration).

After you've done so, make sure to note down the pipeline_id, pipeline_name, project_id, and project_name in the variables below. You'll need these later! 

In [38]:
pipeline_id = "<pipeline_id>"
pipeline_name = "<pipeline_name>"
project_id = "<project_id>"
project_name = "<project_name>"

#### Setup LlamaCloud Client SDK and Framework Client

Here we define both the client (giving us access to low-level client operations) as well as the `LlamaCloudIndex` defined through the framework.

In [39]:
from llama_cloud.client import LlamaCloud

client = LlamaCloud(
    token=os.environ["LLAMA_CLOUD_API_KEY"],
    base_url=os.environ["LLAMA_CLOUD_BASE_URL"]
)

from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
import os

index = LlamaCloudIndex(
  name=pipeline_name, 
  project_name=project_name,
  api_key=os.getenv("LLAMA_CLOUD_API_KEY")
)

## Inserting Documents

Now let's create the custom Document objects. We assume that your pipeline has been created in the last section. Copy the pipeline and project ids into the box below.

We insert one document containing the parsed document text, and another document as a toy example.

#### Inserting Document Objects through the Client SDK

In [25]:
from llama_cloud.types import CloudDocumentCreate


def to_cloud_documents(documents):
    """Converts to Cloud document object.

    The LlamaCloud Client SDK uses a lower-level `CloudDocumentCreate` object,
    similar but different to the `Document` object used in the framework.

    This function converts to a cloud document object.
    """
    return [
        CloudDocumentCreate(
            text=d.get_content(),
            metadata=d.metadata,
            excluded_embed_metadata_keys=d.excluded_embed_metadata_keys,
            excluded_llm_metadata_keys=d.excluded_llm_metadata_keys
        )
        for d in documents
    ]


cloud_documents = to_cloud_documents(documents)
upserted_docs = client.pipelines.upsert_batch_pipeline_documents(pipeline_id, request=cloud_documents)

#### Inserting Document Objects through the Framework Integration

You can also do `index.insert` to directly upload document objects using the types defined by the framework.

In [40]:
# NOTE: the llamaparsed document is already in the right representation
from llama_index.core.schema import Document

for doc in documents:
    index.insert(doc)

#### Validating the Documents

After the documents have been inserted, we can validate that they exist in the pipeline.

In [41]:
pipeline_docs = client.pipelines.list_pipeline_documents(pipeline_id)

print(len(pipeline_docs))

1


#### Deleting the Documents

If you want to reset, you can use the client SDK to delete pipeline documents.

In [ ]:
pipeline_docs = client.pipelines.list_pipeline_documents(pipeline_id)
for doc in pipeline_docs:
    client.pipelines.delete_pipeline_document(pipeline_id, doc.id)
client.pipelines.sync_pipeline(pipeline_id)

## Test RAG

Let's create a sample RAG pipeline through the Python framework, through our `LlamaCloudIndex` (you can also run our lower-level search API through `client.pipelines.run_search`).

In [42]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")
retriever = index.as_retriever(rerank_top_n=5)
query_engine = RetrieverQueryEngine.from_args(
    retriever,
    llm=llm
)
response = query_engine.query("What is the intern test for evaluating generations?")
print(str(response))

The intern test for evaluating generations involves assessing whether an average college student in the relevant major could successfully complete the task given the same input and context as the language model. If the student could succeed, the task is considered feasible for the model. If not, the context may need to be enriched or the task simplified. If the task is too complex even after improvements, it may be beyond the capabilities of contemporary language models.


In [43]:
# view sources
for idx, n in enumerate(response.source_nodes):
    print(f"--------- SOURCE NODE {idx} --------")
    print(n.get_content()[:1000] + "....")

--------- SOURCE NODE 0 --------
Also consider checks to ensure that word, item, or sentence counts lie within a range. For other kinds of generation, assertions can look different. [Execution-evaluation](https://www.semanticscholar.org/paper/Execution-Based-Evaluation-for-Open-Domain-Code-Wang-Zhou/1bed34f2c23b97fd18de359cf62cd92b3ba612c3)
 is a powerful method for **evaluating** code-generation, wherein you run the generated code and determine that the state of runtime is sufficient for the user-request.

As an example, if the user asks for a new function named foo; then after executing the agent’s generated code, foo should be callable! One challenge in execution-evaluation is that the agent code frequently leaves the runtime in slightly different form than the target code. It can be effective to “relax” assertions to the absolute most weak assumptions that any viable answer would satisfy.

Finally, using your product as intended for customers (i.e., “dogfooding”) can provide insigh